[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CoolandHot/DATA5709Capstone/blob/main/DATA5709_Capstone_preprocess.ipynb)

# Preprocess data

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import numpy as np
import pickle
import os


input_month = "06" # this global variable is used to download/save file according to the month
data_saveFile_path_prefix = "/content/gdrive/Shareddrives/Education/5709_Capstone/taxi_"

Mounted at /content/gdrive


## Cleaning function define

In [2]:
def request_write(input_str, input_month):
    url = "https://nyc-tlc.s3.amazonaws.com/trip+data/"+input_str+"_tripdata_2019-"+input_month+".csv"
    !wget $url

# final_col_set = ["PULocationID", "DOLocationID", "PU_datetime", "DO_datetime", "taxi", "weekNum", "hour_of_day", "min_cat"]
final_col_set = ["PULocationID", "DOLocationID", "PU_datetime", "DO_datetime", "taxi"]

def time_series_process(input_pd, month):
    # changes apply to the external variables, by reference
    input_pd.PU_datetime = pd.to_datetime(input_pd.PU_datetime).dt.floor('10min') # floor to nearest 10 min
    input_pd.drop(input_pd[(input_pd.PU_datetime.dt.year != 2019) | (input_pd.PU_datetime.dt.month != int(month))].index, inplace=True) # because some file contains more than this month
    input_pd.DO_datetime = pd.to_datetime(input_pd.DO_datetime).dt.floor('10min')
    input_pd.drop(input_pd[(input_pd.DO_datetime.dt.year != 2019) | (input_pd.DO_datetime.dt.month != int(month))].index, inplace=True)
    # input_pd['weekNum'] = input_pd.PU_datetime.dt.day_name()
    # input_pd['hour_of_day'] = input_pd.PU_datetime.dt.hour
    # input_pd['min_cat'] = input_pd.PU_datetime.dt.minute
    input_pd.drop(columns = [col for col in input_pd if col not in final_col_set], inplace = True)

## Preprocess and save on each dataset

Download

In [3]:
request_write("green", input_month)
request_write("yellow", input_month)
request_write("fhvhv", input_month)

--2021-06-10 03:04:39--  https://nyc-tlc.s3.amazonaws.com/trip+data/green_tripdata_2019-06.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.41.236
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.41.236|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43354841 (41M) [text/csv]
Saving to: ‘green_tripdata_2019-06.csv’

green_tripdata_2019 100%[===================>]  41.35M  41.9MB/s    in 1.0s    

2021-06-10 03:04:41 (41.9 MB/s) - ‘green_tripdata_2019-06.csv’ saved [43354841/43354841]

--2021-06-10 03:04:41--  https://nyc-tlc.s3.amazonaws.com/trip+data/yellow_tripdata_2019-06.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.41.236
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.41.236|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 643492154 (614M) [text/csv]
Saving to: ‘yellow_tripdata_2019-06.csv’

yellow_tripdata_201 100%[==========

Green

In [4]:
%%time
taxi_green = pd.read_csv("/content/green_tripdata_2019-"+input_month+".csv")\
                    .rename(columns={'lpep_pickup_datetime':'PU_datetime',"lpep_dropoff_datetime": "DO_datetime"})
taxi_green["taxi"] = "green"
time_series_process(taxi_green, input_month)
taxi_green = taxi_green[final_col_set] # order the columns
taxi_green.to_csv(data_saveFile_path_prefix + input_month + "_all.csv", index=False)
# clean up memory
del taxi_green

CPU times: user 3.74 s, sys: 212 ms, total: 3.95 s
Wall time: 16.3 s


Yellow

In [5]:
%%time
taxi_yellow = pd.read_csv("/content/yellow_tripdata_2019-"+input_month+".csv")\
            .rename(columns={'tpep_pickup_datetime':'PU_datetime',"tpep_dropoff_datetime": "DO_datetime"})
taxi_yellow["taxi"] = "yellow"
time_series_process(taxi_yellow, input_month)
taxi_yellow = taxi_yellow[final_col_set] # order the columns
taxi_yellow.to_csv(data_saveFile_path_prefix + input_month + "_all.csv", mode='a', header=False, index=False)
# clean up memory
del taxi_yellow

CPU times: user 50.7 s, sys: 2.02 s, total: 52.7 s
Wall time: 53.8 s


FHVHV (too large, needs to read in chunks)

In [6]:
%%time
chunksize = 1e6
chunk_list = []
taxi_fhvhv_chunk = pd.read_csv("/content/fhvhv_tripdata_2019-"+input_month+".csv", chunksize=chunksize)
for chunk in taxi_fhvhv_chunk:
    chunk.rename(columns={'pickup_datetime':'PU_datetime',"dropoff_datetime": "DO_datetime", "hvfhs_license_num": "taxi"}, inplace = True)
    chunk["taxi"] = chunk.taxi.map({"HV0003": "uber", "HV0004": "via", "HV0005": "lyft"}, na_action = "ignore")
    time_series_process(chunk, input_month)
    chunk_list.append(chunk)
# clean up memory
del taxi_fhvhv_chunk
tmp = pd.concat(chunk_list)
del chunk_list
tmp = tmp[final_col_set] # order the columns
tmp.to_csv(data_saveFile_path_prefix + input_month + "_all.csv", mode='a', header=False, index=False)
del tmp

CPU times: user 2min 23s, sys: 2.73 s, total: 2min 26s
Wall time: 2min 30s


The resulting file is huge. 

Look like this:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>PULocationID</th>
      <th>DOLocationID</th>
      <th>PU_datetime</th>
      <th>DO_datetime</th>
      <th>taxi</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>250556</th>
      <td>196</td>
      <td>197</td>
      <td>2019-06-16 21:30:00</td>
      <td>2019-06-16 22:00:00</td>
      <td>green</td>
    </tr>
    <tr>
      <th>106848</th>
      <td>181</td>
      <td>89</td>
      <td>2019-06-07 21:20:00</td>
      <td>2019-06-07 21:30:00</td>
      <td>green</td>
    </tr>
    <tr>
      <th>290375</th>
      <td>220</td>
      <td>136</td>
      <td>2019-06-19 15:10:00</td>
      <td>2019-06-19 15:10:00</td>
      <td>green</td>
    </tr>
    <tr>
      <th>110297</th>
      <td>82</td>
      <td>95</td>
      <td>2019-06-08 01:10:00</td>
      <td>2019-06-08 01:20:00</td>
      <td>green</td>
    </tr>
    <tr>
      <th>453035</th>
      <td>42</td>
      <td>42</td>
      <td>2019-06-29 18:00:00</td>
      <td>2019-06-29 18:00:00</td>
      <td>green</td>
    </tr>
  </tbody>
</table>



```
html = taxi_all.head().to_html()
print(html)
```



## Aggregation and save

[Pandas groupby](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/)

In [ ]:
%%time

def write_to_aggregate_file(req_datetime, req_LocationID, file_suffix):
    # agg() returns only the count column with all group names as index, reset_index() split these group names into columns
    taxi_agg_demand = taxi_all.groupby([req_datetime, req_LocationID, "taxi"]).agg(count = ("taxi", "count")).reset_index()
    taxi_agg_demand.to_csv(data_saveFile_path_prefix + file_suffix , index=False)

for input_month in ["07", "08", "09"]:
    taxi_all = pd.read_csv(data_saveFile_path_prefix + input_month + "_all.csv")

    # rewrite
    time_series_process(taxi_all, input_month)
    taxi_all = taxi_all[final_col_set] # order the columns
    taxi_all.to_csv(data_saveFile_path_prefix + input_month + "_all.csv", index=False)

    req_datetime, req_LocationID, file_suffix = 'PU_datetime', 'PULocationID', input_month+"_agg_demand.csv"
    write_to_aggregate_file(req_datetime, req_LocationID, file_suffix)

    req_datetime, req_LocationID, file_suffix = 'DO_datetime', 'DOLocationID', input_month+"_agg_supply.csv"
    write_to_aggregate_file(req_datetime, req_LocationID, file_suffix)

The aggregated table looks like this:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>DOLocationID</th>
      <th>DO_datetime</th>
      <th>taxi</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
      <td>2019-06-01 00:50:00</td>
      <td>lyft</td>
      <td>1</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1</td>
      <td>2019-06-01 01:00:00</td>
      <td>lyft</td>
      <td>1</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1</td>
      <td>2019-06-01 01:20:00</td>
      <td>uber</td>
      <td>1</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1</td>
      <td>2019-06-01 01:40:00</td>
      <td>lyft</td>
      <td>2</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1</td>
      <td>2019-06-01 01:40:00</td>
      <td>uber</td>
      <td>2</td>
    </tr>
  </tbody>
</table>

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>PULocationID</th>
      <th>PU_datetime</th>
      <th>taxi</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
      <td>2019-06-01 00:00:00</td>
      <td>via</td>
      <td>1</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1</td>
      <td>2019-06-01 00:20:00</td>
      <td>via</td>
      <td>2</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1</td>
      <td>2019-06-01 01:00:00</td>
      <td>via</td>
      <td>1</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1</td>
      <td>2019-06-01 01:10:00</td>
      <td>via</td>
      <td>1</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1</td>
      <td>2019-06-01 02:00:00</td>
      <td>via</td>
      <td>1</td>
    </tr>
  </tbody>
</table>